In [1]:
import sys
from pathlib import Path

from dotenv import load_dotenv

In [2]:
project_root = Path.cwd().parent
sys.path.append(str(project_root))

load_dotenv(project_root / '.env')

True

In [3]:
from ingestion.utils.state_serializer import load_state
from ingestion.config import GUIDELINE_PATHS, API_SPEC_PATHS

In [4]:
guideline_state = load_state(GUIDELINE_PATHS["processed_dir"] / "guideline.pkl")
api_spec_state = load_state(API_SPEC_PATHS["processed_dir"] / "api_specification.pkl")

In [5]:
from ingestion.utils.document_processor import process_document

documents = {}
documents["guideline"] = process_document(guideline_state)
documents["api_specification"] = process_document(api_spec_state)

Error processing table summary: not well-formed (invalid token): line 17, column 33
Error processing table summary: not well-formed (invalid token): line 17, column 14
Error processing table summary: mismatched tag: line 11, column 2
Error processing table summary: not well-formed (invalid token): line 10, column 12
Error processing table summary: not well-formed (invalid token): line 20, column 58
Error processing table summary: not well-formed (invalid token): line 13, column 33


In [6]:
target_docs = documents["guideline"]["docs"] + documents["api_specification"]["docs"] + \
    documents["guideline"]["hypo_docs"] + documents["api_specification"]["hypo_docs"]

In [7]:
from ingestion.retrivers import get_retriever

ensemble_retriever = get_retriever(target_docs)

In [8]:
ensemble_retriever.invoke("정보 전송 요구 연장은 언제 가능한가요?")

[Document(metadata={'type': 'image', 'image': PosixPath('/Users/gabriela/src/workspace/rag-agent/resources/processed/guideline/figures/656.png'), 'page': 74, 'source': PosixPath('/Users/gabriela/src/workspace/rag-agent/resources/original/guideline.pdf'), 'id': 656}, page_content='<image>\n<title>\n마이데이터 앱 전송 요구 단계\n</title>\n<summary>\n이 이미지는 금융분야 마이데이터 앱에서 자산 및 상품 전송 요구를 진행하는 단계들을 보여줍니다. 사용자는 정보 제공자 선택, 인증 수단 선택, 개인신용정보 전송 요구 확인 등의 절차를 거치며, 최대 200개 계좌(7천바이트)로 제한된 전송 요구를 수행할 수 있습니다.\n</summary>\n<entities> \n- 마이데이터 앱\n- 정보 제공자\n- 인증 수단\n- 개인신용정보\n- 전송 요구\n</entities>\n\n</image>'),
 Document(metadata={'type': 'page_summary', 'page': 41, 'source': PosixPath('/Users/gabriela/src/workspace/rag-agent/resources/original/guideline.pdf'), 'text': '3장. 마이데이터서비스\n- (정보제공자) 고객의 개인신용정보 전송 요구에 응하여, 전송요구 주체가 고객\n본인임이 확인되면 마이데이터사업자에게 해당 고객의 개인신용정보를 전송\n하여야 한다.\n관련법령\n브 !\n신용정보법 제33조의2(개인신용정보의 전송요구) ⑧ 제1 항에 따라 본인으로부터 개인신용\n정보의 전송요구를 받은 신용정보제공·이용자등은 신용정보주체의 본인 여부가 확인되지\n아니하는 경우 등 대통령령으로 정하는 경우에는 전송요구

In [9]:
from langchain_openai import ChatOpenAI
from ingestion.utils.evaluator import OpenAIRelevanceGrader

groundedness_check = OpenAIRelevanceGrader(
    ChatOpenAI(model="gpt-4o-mini", temperature=0), target="retrieval-question"
).create()

In [10]:
from ingestion.utils.document_processor import clean_retrieved_documents

retrieved_documents = ensemble_retriever.invoke(
    "x-api-tran-id에 대해 알려주세요."
)

cleaned_documents = clean_retrieved_documents(retrieved_documents)

In [11]:
for doc in cleaned_documents:
    print(doc.page_content)
    print("---" * 30)
    print(doc.metadata)
    print("===" * 30, end="\n\n\n")

<table>
<title>
금융분야 마이데이터 표준API 규격
</title>
<summary>
이 표는 금융분야의 다양한 데이터 코드와 그에 대한 설명을 포함하고 있습니다. 각 코드에 따라 특정한 데이터 유형과 관련된 정보를 제공합니다.
</summary>
<entities> 
- 금융보안원
- 웹사이트: www.fsec.or.kr
- 문서 번호: 303
- 코드: A5300, A5301, A5302, A5309, A5310, A5500, A5999, A6100, A6101, A6103, A6105, A6110, A6111, A6114, A6115, A6117, A6118, A6119, A6120, A6150, A6200, A6201, A6300, A6301, A6302, A6303, A6350
</entities>
<data_insights>
- 다양한 데이터 코드(A5300~A6350)와 그 설명이 포함되어 있으며, 각 코드에 따라 특정한 데이터 유형을 정의하고 있습니다.
- 예를 들어, A5300은 상해로 인한 수술을 보장하는 담보를 의미합니다.
- A6100은 질병으로 인한 입원을 보장하는 담보를 나타냅니다.
</data_insights>

</table>
------------------------------------------------------------------------------------------
{'page': 328, 'source': PosixPath('/Users/gabriela/src/workspace/rag-agent/resources/original/api_specification.pdf')}


<table>
<title>
금융분야 마이데이터 표준API 규격 - 10상품 [구매] 분류 코드
</title>
<summary>
이 표는 금융분야 마이데이터의 표준 API 규격에 따른 10가지 상품 구매 분류 코드를 정리한 것입니다. 각 분류는 특정 카테고리와 관련된 내용을 포함하고 있으며, 코드와 함께 제공됩니다.
</s

In [13]:
from ingestion.retrivers import retrieve_and_check

retrieve_and_check("API 스펙 중 aNS는 어떤 것을 뜻하나요?", 
                   ensemble_retriever)

[Document(metadata={'page': 333, 'source': PosixPath('/Users/gabriela/src/workspace/rag-agent/resources/original/api_specification.pdf')}, page_content='<table>\n<title>\n금융분야 마이데이터 표준API 규격\n</title>\n<summary>\n이 표는 질병 관련 금융보안원의 다양한 담당 코드와 그 설명을 나열하고 있습니다. 각 코드에 따라 특정 질병이나 관련된 치료 방법이 명시되어 있습니다.\n</summary>\n<entities> \n담보 코드: 200, 201, 202, 203, 204, 205, 206, 207, 208, 211, 212, 213, 214, 215, 216, 217, 218, 219\n설명: 질병, 7대질병관련, 여성만성질병관련, 부인과질병관련, 여성특정질병관련, 고액치료비특정질병관련, 남성7대질병관련, 국내여행 질병관련, 해외여행 질병관련, 남북한주민왕래 질병치료비, 마라톤질병의료실비, 해외발생질병의료비, 해외연수생 해외발생질병의료비, 비뇨기계 실손의료비, 산과질병관련, 치료비급여, 한방비급여, 출산확장\n</entities>\n<data_insights>\n총 18개의 담당 코드가 있으며, 각 코드에 따라 다양한 질병 및 치료 관련 정보가 제공됩니다. 특히, 7대 질병과 여성 및 남성 관련 질병에 대한 정보가 포함되어 있습니다.\n</data_insights>\n\n</table>'),
 Document(metadata={'page': 30, 'source': PosixPath('/Users/gabriela/src/workspace/rag-agent/resources/original/guideline.pdf'), 'summary': '- 고객이 종합포털을 통해 마이데이터사업자 외 기관에 개인신용정보 전송 요구\n- 정보제공자는 거점중계기관을 통해 개인신용정보 전송\n- 절차는 변경 가능, 상세 절차